## Expert Knowledge Worker

In [ ]:
# imports libary

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

d:\Code\Job-Aware-RAG-Agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# declare AI model gpt-4o-mini is the most lowest cost model

MODEL = "gpt-4o-mini"

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

In [ ]:
# Test matching documents

context = {}

documents = glob.glob("knowledge_base/Machine_Learning/*")

for document in documents:
    name = document.split(os.sep)[-1][:-3]
    doc = ""
    with open(document, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc


dict_keys(['association_rule_learning', 'clustering', 'decision_tree', 'dimensionality_reduction', 'gradient_boosting', 'knn', 'linear_regression', 'logistic_regression', 'machine_learning', 'machine_learning_overview', 'naive_bayes', 'random_forest', 'regression_classification', 'reinforcement_learning ', 'supervised_learning', 'svm', 'unsupervised_learning'])


In [21]:
context.keys()

dict_keys(['association_rule_learning', 'clustering', 'decision_tree', 'dimensionality_reduction', 'gradient_boosting', 'knn', 'linear_regression', 'logistic_regression', 'machine_learning', 'machine_learning_overview', 'naive_bayes', 'random_forest', 'regression_classification', 'reinforcement_learning ', 'supervised_learning', 'svm', 'unsupervised_learning'])

In [33]:
system_message = """
You are an expert in AI/ML career guidance. You must answer strictly based on the context provided from the database.
If the context does not contain the necessary information, or if the answer cannot be derived directly from the context,
you must respond: "I do not have information in the database to answer this question."

Rules:
- Do not invent, guess, or hallucinate.
- Do not rely on prior knowledge outside the provided context.
- Provide brief, accurate answers only.
- If the context is empty or irrelevant, say you don't have the data.
- All answers must be in Vietnamese.
"""


In [23]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context          

In [26]:
get_relevant_context("what about decision_tree?")

['---\ntitle: "Decision Tree"\ndescription: "Thuật toán Decision Tree trong Machine Learning."\ntags: ["Machine Learning", "Decision Tree"]\n---\n\n# Decision Tree\n\n## 1. Khái niệm\n\nDecision Tree là thuật toán thuộc nhóm **Supervised Learning**, dùng cho cả **Classification** và **Regression**. Mô hình hoạt động bằng cách chia dữ liệu thành các nhánh dựa trên điều kiện, tạo thành cấu trúc giống cây gồm **node**, **branch**, và **leaf**.\n\n---\n\n## 2. Cấu trúc cây quyết định\n\n* **Root Node**: điểm bắt đầu, chứa toàn bộ dữ liệu.\n* **Internal Node**: nút kiểm tra điều kiện (feature).\n* **Leaf Node**: nút kết luận → class label hoặc giá trị dự đoán.\n* **Branch**: đường nối thể hiện lựa chọn theo điều kiện.\n\n---\n\n## 3. Cách hoạt động\n\n1. Chọn feature tốt nhất để chia dữ liệu.\n2. Tạo một node với điều kiện rẽ nhánh.\n3. Lặp lại cho từng nhánh đến khi đạt điều kiện dừng.\n4. Node cuối trở thành leaf node.\n\n---\n\n## 4. Tiêu chí chọn feature (Split Criteria)\n\n### Đối với 

In [ ]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if not relevant_context:
        message += "\n\n[CONTEXT NOT FOUND IN DATABASE]"
    
    message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
    for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [29]:
print(add_context("what about decision tree?"))

what about decision tree?


In [30]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [32]:
view = gr.ChatInterface(chat).launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
